In [9]:
import skimage
import torch
import numpy as np
from pytorch3d.renderer import TexturesVertex
from pytorch3d.structures import Meshes
from pytorch3d.vis.plotly_vis import AxisArgs, plot_batch_individually, plot_scene

def get_torch3dmesh(vertices, triangles, device):
    faces_idx = torch.LongTensor(triangles).to(device)  # faces.verts_idx.to(device)
    verts = torch.tensor(vertices, dtype=torch.float).to(device)  # verts.to(device)

    center = verts.mean(0)
    verts = verts - center
    scale = max(verts.abs().max(0)[0])
    verts = verts / scale
    # Initialize each vertex to be white in color.
    verts_rgb = torch.ones_like(verts)[None]  # (1, V, 3)
    verts_rgb[:, :, 0] = 0.7
    verts_rgb[:, :, 1] = 0.6
    verts_rgb[:, :, 2] = 0.9
    textures = TexturesVertex(verts_features=verts_rgb.to(device))

    # Create a Meshes object
    mesh = Meshes(
        verts=[verts.to(device)],   
        faces=[faces_idx.to(device)],
        textures=textures
    )

    # mesh = Meshes(verts=[verts],
    #               faces=[faces_idx])
    return verts, mesh

def get_vert_mesh(file_path):
    #mesh = pymesh.load_mesh(file_path)
    mesh = trimesh.load(file_path)
    verts, mesh = get_torch3dmesh(mesh.vertices, mesh.faces, 'cpu')
    return verts, mesh

In [10]:
a = np.load('MOF-5.npy')

new_verts, new_faces, normals, _ = skimage.measure.marching_cubes(a, level=0.02)

new_faces = new_faces.copy()
new_verts = new_verts.copy()


_, new_mesh = get_torch3dmesh(new_verts, new_faces, 'cpu')

# plot batch of meshes in different subplots
fig = plot_scene({
    "mesh_with_pbc": {
        "pred": new_mesh
    }
}
)
fig.update_layout(height=1000, width=1000)
fig.show()

#np.save('MOF-5_periodic', new_voxels)
